In [27]:
# nejdříve jsme si otevřely náš soubor pomocí knihovny pandas, odstranily záporné skóre a null hodnoty
# skóre jsme vynásobily 2 abychom získaly celé číslo

import pandas as pd
data = pd.read_csv('soubory/Cukrarny_all_v3_20231111.csv')

positive_data = data[data['score'] != -1]
positive_data.loc[:, 'score'] = positive_data['score'] * 2

In [28]:
#rozdělily jsme data na skóre, které budeme chtít predikovat a na sloupce, které budou vloženy do našeho modelu (informace o poloze)
y = positive_data.score

positive_features = ['gps_latitude', 'gps_longitude', 'zip']

X = positive_data[positive_features]



In [29]:
# z dat jsme oddělily 1/4 na které jsme pak testovaly úspěšnost našeho modelu
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25, random_state = 0)

In [30]:
# pro vytvoření modelu jsme použily rozhodovací strom DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier


our_model = DecisionTreeClassifier(random_state=0)
our_model.fit(train_X, train_y)
prediction = our_model.predict(val_X)


In [31]:
# zde jsem si vyhodnotily úspěšnost našeho modelu
# Ve výsledné matici je vidět, že skóre není možné předpovědět jen na základě polohy cukrárny, u fungujícího modelu by výsledky byly ideálně na uhlopříčce z levého horního rohu do dolního pravého. Takhle nám model u většiny cukráren predikoval nejvyšší hodnocení.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
cm = confusion_matrix(val_y, prediction)
print(cm)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = our_model, X = train_X, y = train_y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100)) #přesnost modelu
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)) #směrodatná odchylka
print(f"Mean Absolute error: {mean_absolute_error(prediction, val_y)}") #střední absolutní chyba

[[ 0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  1  1  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  2  1]
 [ 0  0  0  0  0  0  0  0  2  4]
 [ 0  0  0  0  0  1  0  2  5 11]
 [ 0  1  1  0  1  0  2  2 17 17]
 [ 0  3  1  0  2  1  2 10 14 21]]
Accuracy: 32.69 %
Standard Deviation: 7.31 %
Mean Absolute error: 1.7196969696969697


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(
